**Word Vectors**

In natural language processing, we represent each word as a vector consisting of numbers. The vector encodes the meaning of the word. These numbers (or weights) for each word are learned using various machine learning models, which we will explore in more detail later in this specialization. Rather than make you code the machine learning models from scratch, we will show you how to use them. In the real world, you can always load the trained word vectors, and you will almost never have to train them from scratch.

We will:

1. Predict analogies between words.

2. Use PCA to reduce the dimensionality of the word embeddings and plot them in two dimensions.

3. Compare word embeddings by using a similarity measure (the cosine similarity).

4. Understand how these vector space models work.

**Predict the Countries from Capitals**

We have illustrated the word analogies by finding the capital of a country from the country. We have changed the problem a bit in this part of the assignment. You are asked to predict the countries that corresponds to some capitals. You are playing trivia against some second grader who just took their geography test and knows all the capitals by heart. Thanks to NLP, you will be able to answer the questions properly. In other words, you will write a program that can give you the country by its capital. That way you are pretty sure you will win the trivia game. We will start by exploring the data set.


In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from utils import get_vectors

In [2]:
import numpy as np


def get_vectors(embeddings, words):
    """
    Input:
        embeddings: a word 
        fr_embeddings:
        words: a list of words
    Output: 
        X: a matrix where the rows are the embeddings corresponding to the rows on the list
        
    """
    m = len(words)
    X = np.zeros((1, 300))
    for word in words:
        english = word
        eng_emb = embeddings[english]
        X = np.row_stack((X, eng_emb))
    X = X[1:,:]
    return X

In [4]:
data = pd.read_csv('capitals.txt', delimiter = ' ')
data.columns = ['city1', 'country1', 'city2', 'country2']
data.head(5)

,city1,country1,city2,country2
0,Athens,Greece,Bangkok,Thailand
1,Athens,Greece,Beijing,China
2,Athens,Greece,Berlin,Germany
3,Athens,Greece,Bern,Switzerland
4,Athens,Greece,Cairo,Egypt


In [6]:
pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB 1.3 MB/s eta 0:00:19
   ---------------------------------------- 0.1/24.0 MB 653.6 kB/s eta 0:00:37
   ---------------------------------------- 0.1/24.0 MB 1.2 MB/s eta 0:00:20
   ---------------------------------------- 0.2/24.0 MB 1.3 MB/s eta 0:00:20
    --------------------------------------- 0.3/24.0 MB 1.4 MB/s eta 0:00:17
    --------------------------------------- 0.5/24.0 MB 1.6 MB/s eta 0:00:16
   - -------------------------------------- 0.7/24.0 MB 2.1 MB/s eta 0:00:12
   - -------------------------------------- 0.9/24.0 MB 2.5 MB/s eta 0:00:10
   -- ------------------------------------- 1.2/24.0 MB 2.8 MB/s eta 0:00:09
   -- ------------------------------------- 1.5/24.0 MB 3.1 MB/s eta 0:00:08
   -- ------------------------------------- 1.8/24.0 MB 3.4 MB/s eta 0:00:07
   --- ------------------------------------ 2.1/24.0 MB 3.7 MB/s eta 0:00:06
   -


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import nltk
from gensim.models import keyedVectors

ImportError: cannot import name 'keyedVectors' from 'gensim.models' (c:\Users\Mit\Documents\PythonProjects\ENV\DLogger\lib\site-packages\gensim\models\__init__.py)